In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/ethereum-cryptocurrency-historical-dataset/Ethereum Historical Data.csv")
df.head()

In [ ]:
df["Vol."].value_counts()

In [ ]:
## We should change M and K to 1.000 and 1.000.000
df["Vol."] = df["Vol."].replace("-",np.nan)
df.dropna(inplace=True)
df["Vol."] = (df["Vol."].replace(r'[KM]+', '', regex=True).astype(float) * \
              df["Vol."].str.extract(r'[\d\.]+([KM]+)', expand=False)
                
             .replace(['K','M'], [10**3, 10**6]).astype(int))

In [ ]:
# Our columns should not contain , because python doesn't understand it as thousand parser
df["Price"]=df["Price"].str.extract("([0-9,]+\.?[0-9]+)").replace(',','', regex=True).astype("float")
df["Open"]=df["Open"].str.extract("([0-9,]+\.?[0-9]+)").replace(',','', regex=True).astype("float")
df["High"]=df["High"].str.extract("([0-9,]+\.?[0-9]+)").replace(',','', regex=True).astype("float")
df["Low"]=df["Low"].str.extract("([0-9,]+\.?[0-9]+)").replace(',','', regex=True).astype("float")
df["Change %"] = df["Change %"].str.extract("([-]?[0-9]+\.?[0-9]+)").astype("float")

In [ ]:
df["Date"].sample(12)

In [ ]:
## It's all about date in stock exchange observations so we should make date our index
df['Date'] =  pd.to_datetime(df['Date'], infer_datetime_format=True)
df.set_index('Date',inplace=True)
df.head()

In [ ]:
df.corr()

In [ ]:
## I think this part needs some knowledge about cryptocurrency and stock exchange domain but here is what I thought while doing this:
# y is price of the day before current day 
# x is change between current day and day before
# z is difference between yesterday price minus open price, multiplied by vol because I thought that instant drops and rises should affect the sales of Ethereum
# gap is obvious tho : gap between highest sale and lowest
## After doing these feature engineering we should take only the columns with correlation value between 0.8-0.2
df["gap"] = df["High"] - df["Low"] 
df["y"] = ((df["Price"] * df["Change %"]) + df["Price"] )
df["x"] = ((df["Price"] - df["y"] * df["Change %"])) 
df["z"] = (df["y"]- df["Open"]  * df["Vol."])
df.drop(["High","Low","Open","Vol.","Change %"],axis=1,inplace=True)
df.head()

In [ ]:
df.corr()

In [ ]:
## Now we should look at the skewness and distrubition of data that's very important for us. If data is not normally distributed we should log or sqrt features individually 
import seaborn as sns
sns.distplot(df["gap"])
df["gap"] = np.log(df["gap"]).replace(-np.inf,0);

In [ ]:
sns.distplot(df["gap"]);

In [ ]:
df["gap"] = np.log(df["gap"]).replace(-np.inf,0)
sns.distplot(df["gap"]);

In [ ]:
sns.distplot(df["x"])
df["x"] = np.sqrt(df["x"]).replace(-np.inf,0);

In [ ]:
sns.distplot(df["x"]);

In [ ]:
df["Price"] = np.sqrt(df["Price"]).replace(-np.inf,0)
sns.distplot((df["Price"]));

In [ ]:
!pip install pycaret
from pycaret.regression import *
exp_name = setup(data=df,target="Price")
best_model = compare_models()